In [1]:
pip install pytelegrambotapi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install mtranslate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import telebot
import requests
from mtranslate import translate  # Импорт функции перевода из библиотеки mtranslate

class NorrisBot:
    def __init__(self, token):
        # Инициализация бота с заданным токеном
        self.bot = telebot.TeleBot(token)
        self.register_handlers()  # Регистрация обработчиков команд

    def register_handlers(self):
        # Обработчик команды /start
        @self.bot.message_handler(commands=['start'])
        def send_welcome(message):
            # Создание интерактивной клавиатуры с категориями
            markup = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True)
            categories = ["/animal", "/career", "/celebrity", "/dev", "/fashion", "/food", "/history", "/money", "/movie", "/music"]
            markup.add(*[telebot.types.KeyboardButton(category) for category in categories])
            # Отправка приветственного сообщения с клавиатурой
            self.bot.reply_to(message, "Привет! Я бот с шутками Чака Норриса. Выбери категорию шутки:", reply_markup=markup)

        # Обработчик команд, начинающихся с /
        @self.bot.message_handler(func=lambda message: message.text.startswith('/'))
        def get_joke_by_category(message):
            category = message.text[1:]  # Извлечение категории из сообщения (/category)
            response = requests.get(f'https://api.chucknorris.io/jokes/random?category={category}')
            if response.status_code == 200:
                joke = response.json()['value']  # Извлечение шутки из ответа API
                translated_joke = self.translate_to_russian(joke)  # Перевод шутки
                self.bot.reply_to(message, translated_joke)  # Отправка переведенной шутки
            else:
                self.bot.reply_to(message, "Извините, не удалось получить шутку. Попробуйте позже.")

        # Обработчик всех остальных сообщений
        @self.bot.message_handler(func=lambda message: True)
        def echo_all(message):
            # Ответ на все остальные сообщения
            self.bot.reply_to(message, "Извините, я могу отвечать только на выбранные категории шуток. Для продолжения нажмите /start.")

    def translate_to_russian(self, text):
        translated = translate(text, 'ru')  # Перевод текста на русский
        return translated

    def start(self):
        self.bot.polling()  # Запуск бота

bot = NorrisBot('YOU TOKEN')
bot.start()